# Introduction
- This introduction is the easiest way for Python folks to play with Percolate
- We provider the Postgres client using the Docker compose connection parameteres as defaults
- We can learn how to register entities after which everyone else becomes possible

In [1]:
import requests
#requests.get('http://www.percolationlabs.ai').content

In [2]:
import percolate
from percolate.services import PostgresService
pg = PostgresService()

## Registering entities which are Agents is the key way to use Percolate
- entity models describe agents completely
- Percolate assumes agents are serializable in the "Agent" model which has structured response, sytem prompt and external function refs
- We register Entities as tables that can store data, as Agents that can allow interaction with data
- Agents are added as structured tables but other indexes are added too; embeddings on fields that need them and graph node registry

In [4]:
from percolate.models.p8 import Project, Agent, ModelField, LanguageModelApi, Function, Session, AIResponse, ApiProxy

#we register the core models - these are added by scripts in install anyway but it illustrates for 'bring your own'
for model in [Agent]:
    repo = pg.repository(model)
    repo.register(register_entities=True)

2025-01-27 10:52:14.677 | DEBUG    | percolate.services.PostgresService:register:147 - Created table p8."ApiProxy"
2025-01-27 10:52:14.820 | DEBUG    | percolate.services.PostgresService:register:161 - Created embedding table - p8_embeddings."p8_ApiProxy_embeddings"
2025-01-27 10:52:14.821 | DEBUG    | percolate.services.PostgresService:register:167 - Updating model fields
2025-01-27 10:52:14.888 | DEBUG    | percolate.services.PostgresService:register:170 - Adding the model agent
2025-01-27 10:52:15.110 | INFO     | percolate.services.PostgresService:register:178 - Entity registered


## Percolate configures Langauge models and assumes we can load tokens from the env
- While not recommended in production, for simplicity you might add tokens locally in your database
- The example below registers unique model names - the name is sometimes of the from provider-model if you want to select an API

In [5]:
"""these are stored in the database by default and you can add your own
in the python client your API token is used
in the database you can save the token but the extension can also load it from the environment if its maintained on the database cluster
"""
from percolate.models.p8 import sample_models
sample_models

[LanguageModelApi(name='gpt-4o-2024-08-06', id='b8d1e7c2-da1d-5e9c-96f1-29681dc478e5', model='gpt-4o-2024-08-06', scheme='openai', completions_uri='https://api.openai.com/v1/chat/completions', token_env_key='OPENAI_API_KEY', token=None),
 LanguageModelApi(name='gpt-4o-mini', id='8ecd0ec2-4e25-5211-b6f0-a049cf0dc630', model='gpt-4o-mini', scheme='openai', completions_uri='https://api.openai.com/v1/chat/completions', token_env_key='OPENAI_API_KEY', token=None),
 LanguageModelApi(name='cerebras-llama3.1-8b', id='f921494d-e431-585e-9246-f053d71cc4a3', model='llama3.1-8b', scheme='openai', completions_uri='https://api.cerebras.ai/v1/chat/completions', token_env_key='CEREBRAS_API_KEY', token=None),
 LanguageModelApi(name='groq-llama-3.3-70b-versatile', id='de029bd1-5adb-527d-b78b-55f925ee4c78', model='llama-3.3-70b-versatile', scheme='openai', completions_uri='https://api.groq.com/openai/v1/chat/completions', token_env_key='GROQ_API_KEY', token=None),
 LanguageModelApi(name='claude-3-5-sonne

## Adding APIs
- Percolate registers apis and functions 
- for example a freely available test api is at https://petstore.swagger.io/#/pet/findPetsByStatus
- we add this to percolate below

In [1]:
from percolate.utils.ingestion import add 
add.add_api('swagger_test', 'https://petstore.swagger.io/v2/swagger.json', verbs='get')

2025-01-27 12:13:15.414 | DEBUG    | percolate.utils.ingestion.add:add_api:37 - Added api uri='https://petstore.swagger.io/v2/swagger.json'


In [2]:
import percolate as p8
from percolate.models.p8 import Function

functions = [Function(**f) for f in p8.repository(Function).select()]
"""filter by the group"""
functions

[Function(name='get_pet_findByStatus', id='5bb40e3b-b0c8-e5c9-9480-904ea955717d', key='findPetsByStatus', verb='get', endpoint='/pet/findByStatus', description='Multiple status values can be provided with comma separated strings', function_spec={'name': 'get_pet_findByStatus', 'description': 'Multiple status values can be provided with comma separated strings', 'parameters': {'type': 'object', 'properties': {'status': {'type': 'array', 'description': 'Status values that need to be considered for filter', 'enum': ['available', 'pending', 'sold']}}, 'required': ['status']}}, proxy_uri='https://petstore.swagger.io/v2'),
 Function(name='get_pet_findByTags', id='41fcfd9d-e0ad-ffdc-58ac-33a5f8cd431d', key='findPetsByTags', verb='get', endpoint='/pet/findByTags', description='Multiple tags can be provided with comma separated strings. Use tag1, tag2, tag3 for testing.', function_spec={'name': 'get_pet_findByTags', 'description': 'Multiple tags can be provided with comma separated strings. Use

#### any database loaded function can be called and you can use the metadata to know how to call it(as can an llm)

In [7]:
f = functions[0]
f.function_spec

{'name': 'get_pet_findByStatus',
 'description': 'Multiple status values can be provided with comma separated strings',
 'parameters': {'type': 'object',
  'properties': {'status': {'type': 'array',
    'description': 'Status values that need to be considered for filter',
    'enum': ['available', 'pending', 'sold']}},
  'required': ['status']}}

In [10]:
f(status='available')

[{'id': 9223372036854775141,
  'category': {'id': 3, 'name': 'Smith'},
  'name': 'doggie',
  'photoUrls': [],
  'tags': [],
  'status': 'available'},
 {'id': 9223372036854775142,
  'category': {'id': 3, 'name': 'Smith'},
  'name': 'doggie',
  'photoUrls': [],
  'tags': [],
  'status': 'available'},
 {'id': 9223372036854775143,
  'category': {'id': 0, 'name': 'string'},
  'name': 'doggie',
  'photoUrls': ['string'],
  'tags': [{'id': 0, 'name': 'string'}],
  'status': 'available'},
 {'id': 9223372036854775144,
  'category': {'id': 0, 'name': 'string'},
  'name': 'doggie',
  'photoUrls': ['string'],
  'tags': [{'id': 0, 'name': 'string'}],
  'status': 'available'},
 {'id': 9223372036854775145,
  'category': {'id': 0, 'name': 'string'},
  'name': 'doggie',
  'photoUrls': ['string'],
  'tags': [{'id': 0, 'name': 'string'}],
  'status': 'available'},
 {'id': 9223372036854775146,
  'category': {'id': 0, 'name': 'string'},
  'name': 'doggie',
  'photoUrls': ['string'],
  'tags': [{'id': 0, 'n

### the function manager is responsible for finding functions in the database but you can also ref external functions on your agent
under the hood if any agent either asks to load a function or if the function is added as a reference in it's `get_functions` the funciton manager is used to load the function into context
```python
from percolate.services import FunctionManager
f = FunctionManager()
f.add_functions_by_key('get_pet_findByStatus')
```

In [1]:
import percolate as p8
from pydantic import BaseModel,Field
import typing
from percolate.models import DefaultEmbeddingField

class MyFirstAgent(BaseModel):
    """You are an agent that provides the information you are asked and a second random fact"""
    #because it has no config it will save to the public database schema
    
    name: str = Field(description="Task name")
    #the default embedding field just settgs json_schema_extra.embedding_provider so you can do that yourself
    description:str = DefaultEmbeddingField(description="Task description")
    
    @classmethod
    def get_model_functions(cls):
        """i return a list of functions by key stored in the database"""
        return {
            'get_pet_findByStatus': "a function i used to look up petes based on their status",
            'p8_about' : 'a "native" database function that gives me general information about percolate'
        }

 

In [2]:
from percolate.services.llm import FunctionCall

FunctionCall(name='get_pet_findByStatus', arguments='{"status":"available"}')

FunctionCall(name='get_pet_findByStatus', arguments={'status': 'available'})

In [3]:
r = p8.Agent(MyFirstAgent)
r

2025-01-27 14:25:57.443 | DEBUG    | percolate.services.FunctionManager:add_function:21 - adding function: <bound method ModelRunner.help of Runner(public.MyFirstAgent)>
2025-01-27 14:25:57.450 | DEBUG    | percolate.services.FunctionManager:add_function:24 - added function help
2025-01-27 14:25:57.476 | DEBUG    | percolate.services.FunctionManager:add_function:24 - added function get_pet_findByStatus
2025-01-27 14:25:57.477 | DEBUG    | percolate.services.FunctionManager:add_function:21 - adding function: <bound method ModelRunner.get_entities of Runner(public.MyFirstAgent)>
2025-01-27 14:25:57.479 | DEBUG    | percolate.services.FunctionManager:add_function:24 - added function get_entities
2025-01-27 14:25:57.480 | DEBUG    | percolate.services.FunctionManager:add_function:21 - adding function: <bound method ModelRunner.search of Runner(public.MyFirstAgent)>
2025-01-27 14:25:57.483 | DEBUG    | percolate.services.FunctionManager:add_function:24 - added function search
2025-01-27 14:

Runner(public.MyFirstAgent)

In [4]:
agent = p8.Agent(MyFirstAgent)
agent.run("can you find available pets",limit=2)

2025-01-27 14:25:57.788 | DEBUG    | percolate.services.FunctionManager:add_function:21 - adding function: <bound method ModelRunner.help of Runner(public.MyFirstAgent)>
2025-01-27 14:25:57.791 | DEBUG    | percolate.services.FunctionManager:add_function:24 - added function help
2025-01-27 14:25:57.818 | DEBUG    | percolate.services.FunctionManager:add_function:24 - added function get_pet_findByStatus
2025-01-27 14:25:57.819 | DEBUG    | percolate.services.FunctionManager:add_function:21 - adding function: <bound method ModelRunner.get_entities of Runner(public.MyFirstAgent)>
2025-01-27 14:25:57.820 | DEBUG    | percolate.services.FunctionManager:add_function:24 - added function get_entities
2025-01-27 14:25:57.821 | DEBUG    | percolate.services.FunctionManager:add_function:21 - adding function: <bound method ModelRunner.search of Runner(public.MyFirstAgent)>
2025-01-27 14:25:57.823 | DEBUG    | percolate.services.FunctionManager:add_function:24 - added function search
2025-01-27 14:

"Here are some available pets:\n\n1. **Name:** doggie\n   - **Category:** string\n   - **Photo URL:** [string](string)\n   - **Tags:** string\n\n2. **Name:** Jackie\n   - **Category:** Dogs\n   - **Photo URLs:** [Url1](Url1), [Url2](Url2)\n   - **Tags:** tag name\n\n3. **Name:** Musya\n   - **Photo URL:** [string](string)\n\n4. **Name:** wireless\n   - **Category:** Concord\n   - **Photo URL:** [http://teresa.net](http://teresa.net)\n   - **Tags:** reboot\n\n5. **Name:** digital\n   - **Category:** Sarasota\n   - **Photo URL:** [http://hildegard.net](http://hildegard.net)\n   - **Tags:** copy\n\n6. **Name:** Filo\n   - **Photo URL:** None\n\n7. **Name:** CatTest\n   - **Category:** Cat1\n   - **Photo URL:** [https://petstore.swagger.io/v2/pet.png](https://petstore.swagger.io/v2/pet.png)\n   - **Tags:** tag1\n\n8. **Name:** Postman\n   - **Category:** Eq-dog\n   - **Photo URL:** [URLLink](URLLink)\n   - **Tags:** Doberman\n\n9. **Name:** doggie\n   - **Category:** maxi\n   - **Photo URL

In [9]:
#p8.Agent(MyFirstAgent).run("what can you tell me about percolate")

[{'role': 'function',
  'content': '{"about-these-data": "You called the tool or function `get_pet_findByStatus` and here are some data that may or may not contain the answer to your question - please review it carefully", "data": [{"id": 9223372036854775581, "category": {"id": 0, "name": "string"}, "name": "doggie", "photoUrls": ["string"], "tags": [{"id": 0, "name": "string"}], "status": "available"}, {"id": 9223372036854775582, "category": {"id": 0, "name": "string"}, "name": "doggie", "photoUrls": ["string"], "tags": [{"id": 0, "name": "string"}], "status": "available"}, {"id": 9223372036854775583, "category": {"id": 0, "name": "string"}, "name": "doggie", "photoUrls": ["string"], "tags": [{"id": 0, "name": "string"}], "status": "available"}, {"id": 9223372036854775584, "category": {"id": 0, "name": "string"}, "name": "doggie", "photoUrls": ["string"], "tags": [{"id": 0, "name": "string"}], "status": "available"}, {"id": 9223372036854775585, "category": {"id": 0, "name": "string"},

### Example of how we bootsrap the database by adding the P8 models. 

In [ ]:
def bootstrap(root='../../../extension/sql'):
    """util to generate the sql that we use to setup percolate"""
    
    from percolate.models.p8 import Project, Agent, ModelField, LanguageModelApi, Function, Session, AIResponse, ApiProxy
    from percolate.models.p8 import sample_models
    from percolate.models.utils import SqlModelHelper
    import glob

    root = root.rstrip('/')
    
    models = [ Project, Agent, ModelField, LanguageModelApi, Function, Session, AIResponse, ApiProxy]
        
    """compile the functions into one file"""
    with open(f'{root}/01_add_functions.sql', 'w') as f:
        for sql in glob.glob('../../../extension/sql-staging/p8_pg_functions/**/*.sql',recursive=True):
            with open(sql, 'r') as sql:
                f.write(sql.read())
                f.write('\n\n---------\n\n')

    """add base tables"""            
    with open(f'{root}/02_create_primary.sql', 'w') as f:
        for model in models:
            f.write(pg.repository(model).model_registration_script(secondary=False, primary=True))

    """add the rest"""
    with open(f'{root}/03_create_secondary.sql', 'w') as f:    
        for model in models:
            f.write(pg.repository(model).model_registration_script(secondary=True, primary=False))
        script = SqlModelHelper(LanguageModelApi).get_data_load_statement(sample_models)
        f.write('\n\n-- -----------\n')
        f.write('-- sample models--\n\n')
        
        f.write(script)
bootstrap(root='../../../extension/sql')